In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload
import glob
import pickle
import numpy as np
import os
import astropy.io.fits as fits
import pandas as pd
import pyklip.klip as klip
import pyklip.instruments.MagAO as MagAO
import pyklip.parallelized as parallelized
import pyklip.fakes as fakes
import matplotlib.pyplot as plt
import snr
import sys
import pdb
from tqdm import tqdm
import seaborn as sns
import collapse as c
import warnings
warnings.filterwarnings('ignore')

#####################
# Display and backend
#####################
plt.style.use("default") # Reset to default before layering on any changes
FIG_LARGE = (12, 8) # Default plot figure size for large figures
%config InlineBackend.figure_format = "retina" # Crisp retina display on macs
#%matplotlib qt5 

################
# Theme settings
################
notebook_mode = "paper" # (paper | dark)
sns.set(palette="colorblind", color_codes=True, context="talk")

if notebook_mode.lower() == "paper":
    tick_params = {
        "xtick.top":True,
        "xtick.direction":"in",
        "ytick.right":True,
        "ytick.direction":"in",
        "axes.spine.right":True
    }
    sns.set_style("ticks", tick_params)
    params = {
        "axes.formatter.limits":(-3, 7),
        "xtick.major.size":14,
        "ytick.major.size":14,
        "xtick.minor.visible":True,
        "ytick.minor.visible":True,
    }
    plt.rcParams.update(params)

elif notebook_mode.lower() == "dark":
    plt.style.use("cyberpunk")

else:
    plt.style.use("default")

 ### Finding the Best KLIP parameters from the parameter explorer
 
This notebook does a basic collapse of a parameter explorer fits object and identifies the best parameters based on **peak snr, average snr, contrast, and a combination of these three metrics**. The parameter explorer is a multidimensional array with the results of a grid search of pyKLIP annuli, movement and KL Mode values as shown below:

<img src="pe_example.png" width="300" height="340">

In order to reduce its dimensionality to determined the absolute best parameters, we can do the following:

**Step 1:** The first thing we need to do is create a `PECollapser` object from the module collapse.py. This class takes as inputs: the filepath to the parameter explorer file, the name you want to assign to your data, the planet radius in pixels (ra), the planet position angle (pa), and your chosen highpass filter size in pixels (we usually go with 1/2 the FWHM). It also allows you to choose how you want to handle the planet dimension of the parameter explorer if you injected more than one planet i.e. if you want to mean or median collapse in that dimension.

**Step 2:** Next, we generate the result from collapsing across planets (*further functionality added later*). The output is a dictionary, which I'm going to turn into a pandas dataframe to make things slightly easier. 

In [2]:
# Set input variables
filepath = 'pes/paramexplore_HD142527_27Apr18_Cont_0pctcut_FAKES_a1-25x1m0-24x1s1iwa3_hp1.5-KLmodes-all.fits'
dataname = 'HD142527_27Apr18'
ra, pa, highpass, iwa = 5.8, 55, 1.5, 3


# Instantiate the PECollapser object
collapser = c.PECollapser(filepath, handle_planets='mean', dataname = dataname, ra = ra, pa = pa, hp = highpass, iwa = iwa)

# Get the result
result = pd.DataFrame(collapser.collapse_across_planets(), index = [0])

In [3]:
result

,Dataname,RA,PA,highpass,IWA,PeakSNR_annuli,PeakSNR_movement,PeakSNR_kl,AvgSNR_annuli,AvgSNR_movement,AvgSNR_kl,Contrast_annuli,Contrast_movement,Contrast_kl,Avg_annuli,Avg_move,Avg_kl
0,HD142527_27Apr18,5.8,55,1.5,3,11,11,8,22,2,6,11,1,6,22,2,6


In [ ]:
filelist = glob.glob('data/27Apr18/*.fits')
dataset = MagAO.MagAOData(filelist)
dataset.IWA = iwa
outputdir = 'output'
parallelized.klip_dataset(dataset, outputdir=outputdir, fileprefix=dataname, 
                            annuli=result['PeakSNR_annuli'][0], subsections=1, movement=result['PeakSNR_movement'][0], numbasis=[1,2,3,4,5,10,25,50,100], calibrate_flux=False, 
                            mode="ADI", highpass = highpass, time_collapse='mean', verbose = True)



reading data, num files:  500
Begin align and scale images for each wavelength
Wavelength 0.656 with index 0 has finished align and scale. Queuing for KLIP
Total number of tasks for KLIP processing is 11
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 reference PSFs available for minmove=11, skipping...
less than 1 refe

In [ ]:
klipped = fits.getdata('output/HD142527_27Apr18-KLmodes-all.fits')

In [ ]:
plt.imshow(klipped[0])